<a href="https://githubtocolab.com/Astolfo2332/HyM_prediction/blob/main/0.2_Limpieza_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# Limpieza de datos

In [ ]:
from google.colab import files

files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
'chmod 600 /root/.kaggle/kaggle.json' #Para proteger la información

In [ ]:
!kaggle competitions download -c h-and-m-personalized-fashion-recommendations -f articles.csv
!kaggle competitions download -c h-and-m-personalized-fashion-recommendations -f customers.csv
!kaggle competitions download -c h-and-m-personalized-fashion-recommendations -f transactions_train.csv
! mkdir Data
! unzip transactions_train.csv.zip -d Data
! unzip customers.csv.zip -d Data
! unzip articles.csv.zip -d Data


## 1. Generación de datos faltantes

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn import preprocessing as ps
from difflib import SequenceMatcher
import csv

En este caso vamos a generar 5% de datos faltantes en las columnas de interés, así que aleatoriamente se desorganizan los datasets antes de unirlos en uno y se eliminaran en parte de transacciones el 5% de artículos comprados, en clientes; la edad y finalmente en artículos; el tipo de articulo.

In [23]:
client=pd.read_csv("Data/customers.csv")

In [24]:
sells=pd.read_csv("Data/transactions_train.csv")

In [25]:
articles=pd.read_csv("Data/articles.csv")

In [15]:
def makeitrandom(client,sells,articles):
    client=client.sample(frac=1) #Desorganizamos las filas
    sells=sells.sample(frac=1)
    articles=articles.sample(frac=1)
    client=client.reset_index(drop=True) #Reiniciamos los index
    sells=sells.reset_index(drop=True)
    articles=articles.reset_index(drop=True)
    return client,sells,articles

In [26]:
client,sells,articles=makeitrandom(client,sells,articles)

In [17]:
f=0.05
n_client=int(len(client)*f)
n_sells=int(len(sells)*f)
n_articles=int(len(articles)*f)

In [27]:
client.loc[:n_client,"age"]=np.nan
sells.loc[:n_sells,"article_id"]=np.nan
articles.loc[:n_articles,"product_type_name"]=np.nan

In [28]:
client,sells,articles=makeitrandom(client,sells,articles)

In [20]:
def faltap(data):
    total      = data.isnull().sum().sort_values(ascending=False)
    percent    = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending=False)
    missing  = pd.concat([total,percent],axis=1,keys=["Total","Percent"])
    return missing

In [320]:
faltap(client)

,Total,Percent
Active,907576,66.150819
FN,895050,65.237831
age,83695,6.100308
fashion_news_frequency,16009,1.166854
club_member_status,6062,0.441843
customer_id,0,0.000000
postal_code,0,0.000000


In [321]:
faltap(sells)

,Total,Percent
article_id,1589417,5.000003
t_dat,0,0.000000
customer_id,0,0.000000
price,0,0.000000
sales_channel_id,0,0.000000


In [322]:
faltap(articles)

,Total,Percent
product_type_name,5278,5.000853
detail_desc,416,0.394156
perceived_colour_master_name,0,0.000000
garment_group_name,0,0.000000
garment_group_no,0,0.000000
section_name,0,0.000000
section_no,0,0.000000
index_group_name,0,0.000000
index_group_no,0,0.000000
index_name,0,0.000000


## 2. agrupación de datos y eliminación de variables

En este caso vamos a proceder a primero hacer una correlación con las compras y los artículos de cada cliente, para generar un dataframe, que nos resuma los artículos comprados Por cliente, el dataset al ser tan grande primero se le hará una reducción a las transacciones y una división entre train y test por fechas

In [11]:
a=len(sells)//6
sells=sells.loc[0:a]
sells.index=sells.customer_id.values
del(sells["sales_channel_id"])
sells.head()

,t_dat,customer_id,article_id,price
396b6c81f9709f67e78f13285dd9653503b8938ed1f96bbb274446fc0167e2a7,2019-05-06,396b6c81f9709f67e78f13285dd9653503b8938ed1f96b...,690936003.0,0.050831
235c312c773dbf4294c45b104928f5d4fff5d4b8d8fce167298cf25dbe02a475,2019-10-31,235c312c773dbf4294c45b104928f5d4fff5d4b8d8fce1...,722177008.0,0.016932
cf721968a879821581583ca507b7bef94718feec547dc52b25833898178f6a73,2019-09-10,cf721968a879821581583ca507b7bef94718feec547dc5...,811235001.0,0.032525
ebd83ef9712abc93b59c209c5cb89755c7a8ab6964f345b7090d25e7428ba0d0,2020-08-29,ebd83ef9712abc93b59c209c5cb89755c7a8ab6964f345...,778745009.0,0.033475
963ae6a061e0c6d85c51ec30625997a042fdbf423558b20e6eb9ab82cf63966d,2019-08-08,963ae6a061e0c6d85c51ec30625997a042fdbf423558b2...,805510004.0,0.016932


In [324]:
client.index=client.customer_id.values
del(client["postal_code"])
del(client["customer_id"])
client.head()

,FN,Active,club_member_status,fashion_news_frequency,age
d65c96df9fc6e0d655dd4456f010f3041d935a0bd99b07d162f50ef577a10507,NaN,NaN,ACTIVE,NONE,NaN
fb6bfaa95a8aa39ee354d191266b64b66d2cff4b26dcd0cefe7b62a5549fb7d8,NaN,NaN,ACTIVE,NONE,32.0
f2054c89bb821c1c5de84acb16ff6dc2079c3a697fd1d2ce4a882ad73144b789,1.0,1.0,ACTIVE,Regularly,53.0
093363b125837ef52d1eb63d6a57d693a44173c8c52cbbc47b0b97fb432a84d4,NaN,NaN,ACTIVE,NONE,21.0
902cd4cfb5573f9090e495ec1dfe64e737ed8334dd0ac829dcc3d5b7736c385d,1.0,1.0,ACTIVE,Regularly,52.0


In [325]:
sells=sells.join(client)
sells.head()

,t_dat,customer_id,article_id,price,FN,Active,club_member_status,fashion_news_frequency,age
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,2019-11-28,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,785710001.0,0.024407,NaN,NaN,ACTIVE,NONE,49.0
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,2020-09-05,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601043.0,0.050831,NaN,NaN,ACTIVE,NONE,49.0
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,2018-12-27,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,627759010.0,0.030492,NaN,NaN,ACTIVE,NONE,49.0
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,2019-11-28,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,694736004.0,0.018288,NaN,NaN,ACTIVE,NONE,49.0
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,2019-09-18,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,656719005.0,0.044051,NaN,NaN,ACTIVE,NONE,49.0


In [326]:
sells=sells.reset_index(drop=True)

In [327]:
sells.head()

,t_dat,customer_id,article_id,price,FN,Active,club_member_status,fashion_news_frequency,age
0,2019-11-28,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,785710001.0,0.024407,NaN,NaN,ACTIVE,NONE,49.0
1,2020-09-05,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601043.0,0.050831,NaN,NaN,ACTIVE,NONE,49.0
2,2018-12-27,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,627759010.0,0.030492,NaN,NaN,ACTIVE,NONE,49.0
3,2019-11-28,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,694736004.0,0.018288,NaN,NaN,ACTIVE,NONE,49.0
4,2019-09-18,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,656719005.0,0.044051,NaN,NaN,ACTIVE,NONE,49.0


## 2.1 Relleno de datos vacíos  

En este caso vamos a reemplazar los datos faltantes, en el caso de el articulo, fashion_news, con la moda, el FN y active con 0. Y age con el promedio.

In [328]:
sells["article_id"].fillna(sells.article_id.mode()[0], inplace=True)
sells["FN"].fillna(0, inplace=True)
sells["Active"].fillna(0, inplace=True)
sells["club_member_status"].fillna(sells.club_member_status.mode()[0], inplace=True)
sells["fashion_news_frequency"].fillna(sells.fashion_news_frequency.mode()[0], inplace=True)
sells["age"].fillna(int(sells.age.mean()), inplace=True)

In [329]:
sells.isnull().sum()

t_dat                     0
customer_id               0
article_id                0
price                     0
FN                        0
Active                    0
club_member_status        0
fashion_news_frequency    0
age                       0
dtype: int64

## 2.2 Filtrado de datos y concatenación final

Ahora vamos a filtrar los artículos para las columnas que consideramos afecten más la compra futura, como es el tipo de articulo, color, la sección a la que pertenece (ya que de ahí podríamos inferir el genero del cliente)

In [330]:
sells.index=sells.article_id.values
sells.head()

,t_dat,customer_id,article_id,price,FN,Active,club_member_status,fashion_news_frequency,age
785710001.0,2019-11-28,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,785710001.0,0.024407,0.0,0.0,ACTIVE,NONE,49.0
568601043.0,2020-09-05,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601043.0,0.050831,0.0,0.0,ACTIVE,NONE,49.0
627759010.0,2018-12-27,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,627759010.0,0.030492,0.0,0.0,ACTIVE,NONE,49.0
694736004.0,2019-11-28,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,694736004.0,0.018288,0.0,0.0,ACTIVE,NONE,49.0
656719005.0,2019-09-18,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,656719005.0,0.044051,0.0,0.0,ACTIVE,NONE,49.0


In [331]:
articles=articles[["product_type_name","article_id","colour_group_name","index_group_name","section_name"]]
articles.index=articles.article_id.values
del(articles["article_id"])

In [332]:
sells=sells.join(articles)
sells=sells.reset_index(drop=True)
sells.head()

,t_dat,customer_id,article_id,price,FN,Active,club_member_status,fashion_news_frequency,age,product_type_name,colour_group_name,index_group_name,section_name
0,2018-09-29,001ae5408a043f64bccd32beffe2730151414cbdf18a6e...,108775015.0,0.006763,1.0,1.0,ACTIVE,Regularly,50.0,Vest top,Black,Ladieswear,Womens Everyday Basics
1,2019-05-12,0022a721371d5949d174ecba60346d89a9d6c08c0fba4f...,108775015.0,0.008458,0.0,0.0,ACTIVE,NONE,48.0,Vest top,Black,Ladieswear,Womens Everyday Basics
2,2019-02-26,005b40b50b0884068e374dbc18dfb712f2a9282f12d489...,108775015.0,0.008458,1.0,1.0,ACTIVE,Regularly,51.0,Vest top,Black,Ladieswear,Womens Everyday Basics
3,2018-11-14,00ada42f83a04a8c02035021f31c83989c2d1b65286298...,108775015.0,0.008458,1.0,1.0,ACTIVE,Regularly,22.0,Vest top,Black,Ladieswear,Womens Everyday Basics
4,2019-02-23,00b19d74d6d60c157a9e08e8d83d9b38b1142a32676543...,108775015.0,0.007186,1.0,1.0,ACTIVE,Regularly,31.0,Vest top,Black,Ladieswear,Womens Everyday Basics


In [333]:
sells.index=pd.to_datetime(sells.t_dat)
del(sells["t_dat"])
sells.head()

,customer_id,article_id,price,FN,Active,club_member_status,fashion_news_frequency,age,product_type_name,colour_group_name,index_group_name,section_name
t_dat,,,,,,,,,,,,
2018-09-29,001ae5408a043f64bccd32beffe2730151414cbdf18a6e...,108775015.0,0.006763,1.0,1.0,ACTIVE,Regularly,50.0,Vest top,Black,Ladieswear,Womens Everyday Basics
2019-05-12,0022a721371d5949d174ecba60346d89a9d6c08c0fba4f...,108775015.0,0.008458,0.0,0.0,ACTIVE,NONE,48.0,Vest top,Black,Ladieswear,Womens Everyday Basics
2019-02-26,005b40b50b0884068e374dbc18dfb712f2a9282f12d489...,108775015.0,0.008458,1.0,1.0,ACTIVE,Regularly,51.0,Vest top,Black,Ladieswear,Womens Everyday Basics
2018-11-14,00ada42f83a04a8c02035021f31c83989c2d1b65286298...,108775015.0,0.008458,1.0,1.0,ACTIVE,Regularly,22.0,Vest top,Black,Ladieswear,Womens Everyday Basics
2019-02-23,00b19d74d6d60c157a9e08e8d83d9b38b1142a32676543...,108775015.0,0.007186,1.0,1.0,ACTIVE,Regularly,31.0,Vest top,Black,Ladieswear,Womens Everyday Basics


In [334]:
sells.tail()

,customer_id,article_id,price,FN,Active,club_member_status,fashion_news_frequency,age,product_type_name,colour_group_name,index_group_name,section_name
t_dat,,,,,,,,,,,,
2020-09-16,a6dbf607f1b6b6ce551b9a4abce877ed3db317243f3380...,953763001.0,0.022017,1.0,1.0,ACTIVE,Regularly,53.0,Vest top,Black,Ladieswear,H&M+
2020-09-15,c05c2f6a82ea17370fec2d3fd2c035d51e47a87de7f63c...,953763001.0,0.022017,0.0,0.0,ACTIVE,NONE,35.0,Vest top,Black,Ladieswear,H&M+
2020-09-11,36e14964e2cbbd442d9f0093e0d95cab4a457e707e07d2...,956217002.0,0.059305,1.0,1.0,ACTIVE,Regularly,28.0,Dress,Black,Ladieswear,Womens Trend
2020-09-14,3e8272741c39b3579424da34b2c8b752d2cce5010774d5...,956217002.0,0.059305,1.0,1.0,ACTIVE,Regularly,36.0,Dress,Black,Ladieswear,Womens Trend
2020-09-13,6b05a969d676cf290cd44fc55ab3067d8406fb3e3bbc47...,956217002.0,0.059305,0.0,0.0,ACTIVE,NONE,50.0,Dress,Black,Ladieswear,Womens Trend


## 3 Transformación de variables categóricas a numéricas

In [335]:
var_categoricas = ['customer_id', 'club_member_status',"fashion_news_frequency","product_type_name","colour_group_name","index_group_name","section_name"]
encoder = ps.LabelEncoder()

for i in var_categoricas:
    
    sells[i] = encoder.fit_transform(sells[i])

sells.article_id=sells.article_id.astype(int)
sells["FN"]=sells["FN"].astype(int)
sells.Active=sells.Active.astype(int)
sells.age=sells.age.astype(int)
sells=sells.reset_index(drop=True)
print (sells.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5298055 entries, 0 to 5298054
Data columns (total 12 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   customer_id             int32  
 1   article_id              int32  
 2   price                   float64
 3   FN                      int32  
 4   Active                  int32  
 5   club_member_status      int32  
 6   fashion_news_frequency  int32  
 7   age                     int32  
 8   product_type_name       int32  
 9   colour_group_name       int32  
 10  index_group_name        int32  
 11  section_name            int32  
dtypes: float64(1), int32(11)
memory usage: 262.7 MB
None


Separamos directamente los datos de test y de train

In [336]:
sells

,customer_id,article_id,price,FN,Active,club_member_status,fashion_news_frequency,age,product_type_name,colour_group_name,index_group_name,section_name
0,410,108775015,0.006763,1,1,0,2,50,118,1,2,43
1,523,108775015,0.008458,0,0,0,1,48,118,1,2,43
2,1339,108775015,0.008458,1,1,0,2,51,118,1,2,43
3,2524,108775015,0.008458,1,1,0,2,22,118,1,2,43
4,2577,108775015,0.007186,1,1,0,2,31,118,1,2,43
...,...,...,...,...,...,...,...,...,...,...,...,...
5298050,632125,953763001,0.022017,1,1,0,2,53,118,1,2,18
5298051,728772,953763001,0.022017,0,0,0,1,35,118,1,2,18
5298052,207639,956217002,0.059305,1,1,0,2,28,30,1,2,53
5298053,236131,956217002,0.059305,1,1,0,2,36,30,1,2,53


## 4 Separación de train y test

In [337]:
div=len(sells)//4
train=sells[div:]
test=sells[:div]

In [338]:
train.to_csv("Data/train.csv",index=False)
test.to_csv("Data/test.csv",index=False)

Observemos que si hayan datos en común para a futuro evaluar las métricas

In [2]:
train=pd.read_csv("Data/train.csv")
test=pd.read_csv("Data/test.csv")

In [3]:
a=len(train)//3
p1=train[:a]
p2=train[a:2*a]
p3=train[2*a:]

In [4]:
p1.to_csv("data_set_1.csv",index=False)
p2.to_csv("data_set_2.csv",index=False)
p3.to_csv("data_set_3.csv",index=False)
test.to_csv("data_set_4.cvs",index=False)

In [58]:
train_c=train.customer_id.unique().tolist()
test_c=test.customer_id.unique().tolist()

In [59]:
common=list(set(train_c)&set(test_c))